In [ ]:
import mlflow
import numpy as np
import pandas as pd
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

ModuleNotFoundError: No module named 'wordcloud'

In [2]:
pip install mlflow

  Obtaining dependency information for mlflow from https://files.pythonhosted.org/packages/c7/16/57765db94704f001fe68a9d5545a883a8ccf6bb6629603c20736f2e388c4/mlflow-3.3.2-py3-none-any.whl.metadata
  Obtaining dependency information for mlflow-skinny==3.3.2 from https://files.pythonhosted.org/packages/fb/06/805b94473d6222249b87fa415d047b34db1e6eb24385783b6d1bc89f6ea8/mlflow_skinny-3.3.2-py3-none-any.whl.metadata
  Obtaining dependency information for mlflow-tracing==3.3.2 from https://files.pythonhosted.org/packages/5e/86/396a17af4e994c7ffa65609739baddc17f4436aec9511478816e157a1bda/mlflow_tracing-3.3.2-py3-none-any.whl.metadata
  Obtaining dependency information for Flask<4 from https://files.pythonhosted.org/packages/ec/f9/7f9263c5695f4bd0023734af91bedb2ff8209e8de6ead162f35d8dc762fd/flask-3.1.2-py3-none-any.whl.metadata
  Obtaining dependency information for alembic!=1.10.0,<2 from https://files.pythonhosted.org/packages/39/4a/4c61d4c84cfd9befb6fa08a702535b27b21fff08c946bc2f6139decbf7f

In [7]:
!pip install nltk
!pip install scikit-learn

  Obtaining dependency information for nltk from https://files.pythonhosted.org/packages/4d/66/7d9e26593edda06e8cb531874633f7c2372279c3b0f46235539fe546df8b/nltk-3.9.1-py3-none-any.whl.metadata
  Obtaining dependency information for regex>=2021.8.3 from https://files.pythonhosted.org/packages/b5/25/d64543fb7eb41a1024786d518cc57faf1ce64aa6e9ddba097675a0c2f1d2/regex-2025.9.1-cp312-cp312-macosx_10_13_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 1.1 MB/s eta 0:00:00
  Obtaining dependency information for tqdm from https://files.pythonhosted.org/packages/d0/30/dc54f88dd4a2b5dc8a0279bdd7270e735851848b762aeb1c1184ed1f6b14/tqdm-4.67.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 8.0 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.7/289.7 kB 6.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 

In [4]:
import matplotlib.pyplot as plt

Matplotlib is building the font cache; this may take a moment.


In [17]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import svm
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import pickle
import nltk
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('stopwords')


# Step 1: Import Necessary Libraries and Load Data
df = pd.read_csv('/Users/samharvey/Desktop/MSO_Git/sentiment_analysis/trainng_data/sentiment_train.csv')
df['text'] = df['text'].astype(str).apply(lambda x: x.lower())

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/samharvey/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/samharvey/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/samharvey/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
df = df.dropna(subset=['lable'])

In [19]:
# Step 2: Preprocess the Data
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    tokens = word_tokenize(text)
    tokens = [WordNetLemmatizer().lemmatize(token) for token in tokens if token.isalpha()]
    return " ".join(tokens)
df['text'] = df['text'].apply(preprocess_text)

X_train, X_test, y_train, y_test = train_test_split(df['text'], df['lable'], random_state=42, test_size=0.2, stratify=df['lable'])


In [20]:
# Step 3: Feature Extraction
vectorizer = TfidfVectorizer(max_features=20000, ngram_range=(1, 2))
X_train_counts = vectorizer.fit_transform(X_train)
X_test_counts = vectorizer.transform(X_test)

In [ ]:
with open('tfidf_vectorizer.pkl', 'wb') as file:
    pickle.dump(vectorizer, file)

In [21]:
# Step 4: Train an SVM Classifier
clf = svm.SVC(kernel='linear')
clf.fit(X_train_counts, y_train)
#with open('svm_model.pkl', 'wb') as f:
  #  pickle.dump(clf, f)
#loaded_clf = pickle.load(open('svm_model.pkl', 'rb'))

,C,1.0
,kernel,'linear'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


In [22]:
# Step 5: Evaluate the Model
X_test_counts = vectorizer.transform(X_test)  # Transform X_test into numerical representation

y_pred = clf.predict(X_test_counts)  # Use the transformed data for prediction
from sklearn.metrics import accuracy_score, confusion_matrix
accuracy = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
print(f"Accuracy: {accuracy:.3f}")
print(cm)

Accuracy: 0.834
[[1271  314   30]
 [ 159 2089   84]
 [ 136  222 1387]]


In [ ]:
from sklearn.metrics import classification_report
report = classification_report(y_test, y_pred)
print(report.)

              precision    recall  f1-score   support

    negative       0.81      0.79      0.80      1615
     neutral       0.80      0.90      0.84      2332
    positive       0.92      0.79      0.85      1745

    accuracy                           0.83      5692
   macro avg       0.84      0.83      0.83      5692
weighted avg       0.84      0.83      0.83      5692



In [33]:
#log model and parameters to mlflow
mlflow.set_tracking_uri("http://127.0.0.1:8080")
mlflow.set_experiment("Sentiment Analysis SVM Experiment")
with mlflow.start_run():
    mlflow.log_param("model_type", "SVM")
    mlflow.log_param("kernel", "linear")
    mlflow.log_param("max_features", 20000)
    mlflow.log_param("ngram_range", (1, 2))
    mlflow.log_metric("accuracy", accuracy)

🏃 View run hilarious-moose-407 at: http://127.0.0.1:8080/#/experiments/818784246610975517/runs/1e85877afa2d4737ad7be218b8de38b3
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/818784246610975517


In [ ]:
from sklearn.model_selection import GridSearchCV

# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']}

grid = GridSearchCV(svm.SVC(), param_grid, refit = True, verbose = 3)

# fitting the model for grid search
grid.fit(X_train_counts, y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.659 total time=  47.0s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.650 total time=  41.6s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.657 total time=  42.5s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.657 total time=  42.1s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.652 total time=  41.8s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.574 total time=  46.2s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.573 total time=  46.2s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.574 total time=  47.2s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.583 total time=  46.7s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.569 total time=  45.5s
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.405 total time=  50.2s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)

In [ ]:
grid_predictions = grid.predict(X_test_counts)

# print classification report
print(classification_report(y_test, grid_predictions))

              precision    recall  f1-score   support

    negative       0.82      0.80      0.81      1556
     neutral       0.79      0.88      0.83      2224
    positive       0.91      0.80      0.85      1717

    accuracy                           0.83      5497
   macro avg       0.84      0.83      0.83      5497
weighted avg       0.84      0.83      0.83      5497



In [ ]:
df_predict = pd.read_csv('/content/subs_q27.csv')
predict_text = df_predict['text'].astype(str).apply(lambda x: x.lower())
predict_text = df_predict['text'].apply(preprocess_text)

predict = vectorizer.transform(predict_text)




In [ ]:

prediction = grid.predict(predict)
df_predict['predictions'] = prediction
df_predict



,text,predictions
0,"A huge thank you to all the musicians, those w...",neutral
1,All great,positive
2,although I was unable to attend all the Beetho...,neutral
3,Amazing and talented musicians. We're very luc...,positive
4,Appalling response to Jayson Gillham’s simple ...,neutral
5,Appreciate the free bonus concerts but often s...,neutral
6,Appreciate the student ticket prices,positive
7,As an elderly resident of the Bellarine Penins...,neutral
8,As pensioners we do not have the capacity to m...,neutral
9,"Carmina Burana was my favourite, it was very s...",positive


In [ ]:

df_predict['predictions'] = prediction
df_predict


,text,predictions
0,"A huge thank you to all the musicians, those w...",neutral
1,All great,positive
2,although I was unable to attend all the Beetho...,neutral
3,Amazing and talented musicians. We're very luc...,positive
4,Appalling response to Jayson Gillham’s simple ...,neutral
5,Appreciate the free bonus concerts but often s...,neutral
6,Appreciate the student ticket prices,positive
7,As an elderly resident of the Bellarine Penins...,neutral
8,As pensioners we do not have the capacity to m...,neutral
9,"Carmina Burana was my favourite, it was very s...",positive


In [ ]:
import keras

In [ ]:
label_counts = df['lable'].value_counts()
print(label_counts)

lable
neutral     11118
positive     8582
negative     7781
Name: count, dtype: int64


In [34]:

from sklearn.utils import resample

# Separate majority and minority classes
df_majority = df[df['lable'] == 'neutral']
df_minority = df[df['lable'] == 'negative']
df_middle = df[df['lable'] == 'positive']

# Downsample majority class
df_majority_downsampled = resample(df_majority,
                                 replace=False,    # sample without replacement
                                 n_samples=len(df_minority),     # to match minority class
                                 random_state=123) # reproducible results

# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority, df_middle])

# Display new class counts
print(df_downsampled['lable'].value_counts())


lable
positive    8727
neutral     8074
negative    8074
Name: count, dtype: int64


In [35]:
df_downsampled['text'] = df_downsampled['text'].astype(str).apply(lambda x: x.lower())
df_downsampled['text'] = df_downsampled['text'].apply(preprocess_text)

X_train, X_test, y_train, y_test = train_test_split(df_downsampled['text'], df_downsampled['lable'], random_state=42, test_size=0.2, stratify=df_downsampled['lable'])

vectorizer = TfidfVectorizer(max_features=20000, ngram_range=(1, 2))
X_train_counts = vectorizer.fit_transform(X_train)
X_test_counts = vectorizer.transform(X_test)

In [36]:
clf = svm.SVC(C=10, gamma=0.1, kernel='rbf' )
clf.fit(X_train_counts, y_train)

,C,10
,kernel,'rbf'
,degree,3
,gamma,0.1
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


In [ ]:
from sklearn.model_selection import GridSearchCV

# defining parameter range
param_grid = {'C': [ 10 ],
              'gamma': [0.07,0.08,0.09],
              'kernel': ['rbf']}

grid = GridSearchCV(svm.SVC(), param_grid, refit = True, verbose = 3)

# fitting the model for grid search
grid.fit(X_train_counts, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV 1/5] END ......C=10, gamma=0.07, kernel=rbf;, score=0.826 total time=  22.4s
[CV 2/5] END ......C=10, gamma=0.07, kernel=rbf;, score=0.826 total time=  21.5s
[CV 3/5] END ......C=10, gamma=0.07, kernel=rbf;, score=0.821 total time=  22.3s
[CV 4/5] END ......C=10, gamma=0.07, kernel=rbf;, score=0.831 total time=  21.9s
[CV 5/5] END ......C=10, gamma=0.07, kernel=rbf;, score=0.811 total time=  21.5s
[CV 1/5] END ......C=10, gamma=0.08, kernel=rbf;, score=0.824 total time=  22.2s
[CV 2/5] END ......C=10, gamma=0.08, kernel=rbf;, score=0.826 total time=  22.5s
[CV 3/5] END ......C=10, gamma=0.08, kernel=rbf;, score=0.820 total time=  22.3s
[CV 4/5] END ......C=10, gamma=0.08, kernel=rbf;, score=0.828 total time=  22.2s


KeyboardInterrupt: 

In [ ]:
#X_test_counts = vectorizer.transform(X_test)  # Transform X_test into numerical representation
from sklearn.metrics import classification_report
y_pred = clf.predict(X_test_counts)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    negative       0.83      0.83      0.83      1556
     neutral       0.77      0.86      0.81      1556
    positive       0.91      0.81      0.86      1717

    accuracy                           0.83      4829
   macro avg       0.84      0.83      0.83      4829
weighted avg       0.84      0.83      0.83      4829



In [ ]:
# prompt: save clf model using pickle

import pickle

# ... (Your existing code) ...

# Save the trained model
#with open('svm_model.pkl', 'wb') as f:
  #  pickle.dump(clf, f)

with open('tfidf_vectorizer.pkl', 'wb') as file:
    pickle.dump(vectorizer, file)


In [ ]:
prediction = clf.predict(predict)
df_predict['predictions'] = prediction
df_predict

,text,predictions
0,"A huge thank you to all the musicians, those w...",negative
1,All great,positive
2,although I was unable to attend all the Beetho...,neutral
3,Amazing and talented musicians. We're very luc...,positive
4,Appalling response to Jayson Gillham’s simple ...,negative
5,Appreciate the free bonus concerts but often s...,negative
6,Appreciate the student ticket prices,negative
7,As an elderly resident of the Bellarine Penins...,negative
8,As pensioners we do not have the capacity to m...,positive
9,"Carmina Burana was my favourite, it was very s...",negative


In [ ]:
df_survey = pd.read_csv('/content/survey.csv')
df_survey['text'] = df_survey['text'].astype(str).apply(lambda x: x.lower())
predict_text = df_survey['text'].astype(str).apply(lambda x: x.lower())
predict_text = df_survey['text'].apply(preprocess_text)

predict = vectorizer.transform(predict_text)

In [ ]:
loaded_model = pickle.load(open('/content/svm_model.pkl', 'rb'))

In [ ]:
prediction = loaded_model.predict(predict)
df_survey['label'] = prediction
df_survey

,text,category,label
0,"a huge thank you to all the musicians, those w...",other,neutral
1,and a special thank you to jaime!,Brand,positive
2,all great,brand,positive
3,although i was unable to attend all the beetho...,Programming and concert format,neutral
4,amazing and talented musicians. we're very luc...,brand,positive
...,...,...,...
977,we need video game music themed concerts,Programming and concert format,neutral
978,"where possible or appropriate, live snippets o...",Programming and concert format,neutral
979,would like better accessibility to programs pr...,Programming and concert format,neutral
980,would like to see more mso's engagement with s...,brand,neutral


In [ ]:
df_survey['label'].value_counts()

,count
label,
neutral,677
positive,169
negative,136


In [ ]:
df_survey.to_csv('/content/survey_results.csv')